# Test TitleExtractor

Notebook minimale per testare il TitleExtractor dalla classe retrieve.py

In [1]:
# Import necessari
import sys
import os

# Aggiungi il path corrente per importare retrieve.py
sys.path.append(os.getcwd())

from retrieve import TitleExtractor

/Users/francesco/Desktop/ACCA/rag/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Inizializza il TitleExtractor
title_extractor = TitleExtractor()
print("TitleExtractor inizializzato")
print(f"LLM disponibile: {title_extractor.llm is not None}")

TitleExtractor inizializzato
LLM disponibile: True


In [3]:
# Estrai titoli reali dal database Milvus (stessa logica di complete_search)
from retrieve import DocumentRetriever, build_pdf_index

# Inizializza il retriever per accedere al database
retriever = DocumentRetriever()

# File da cui estrarre i titoli
target_file = "ccnl_commercio_terziario_distribuzione_e_servizi.pdf"  # CCNL
# target_file = "BIS - Regolamento Aziendale.pdf"  # Per test futuro con Regolamento Aziendale

print(f"Estrazione titoli da: {target_file}")
print("-" * 50)

try:
    # Filtra per il file specifico
    filter_expr = f'filename == "{target_file}"'
    
    # Ottieni tutti i chunks per questo file (k alto per prendere tutto)
    docs_scores = retriever.vector_store.similarity_search_with_score(
        query="", k=10000, expr=filter_expr
    )
    
    # Estrai titoli unici dal metadata, filtrando quelli non validi
    seen_titles = set()
    real_titles = []
    
    for doc, score in docs_scores:
        chunk_title = doc.metadata.get("chunk_title", "")
        if (chunk_title and 
            chunk_title not in ["UNKNOWN", "SKIP", "TO_ASSIGN", ""] and 
            chunk_title not in seen_titles):
                real_titles.append(chunk_title)
                seen_titles.add(chunk_title)
    
    print(f"Trovati {len(real_titles)} titoli unici:")
    print()
    
    # Stampa i primi 15 titoli per non sovraccaricare l'output
    for i, title in enumerate(real_titles[:15], 1):
        print(f"{i}. {title}")
    
    if len(real_titles) > 15:
        print(f"... e altri {len(real_titles) - 15} titoli")
    
    # Usa i titoli reali per i test
    sample_titles = real_titles
    
except Exception as e:
    print(f"Errore nell'estrazione dei titoli: {e}")
    print("Uso titoli di esempio come fallback")
    
    sample_titles = []

/Users/francesco/Desktop/ACCA/rag/.venv/lib/python3.11/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


Estrazione titoli da: ccnl_commercio_terziario_distribuzione_e_servizi.pdf
--------------------------------------------------
Trovati 175 titoli unici:

1. Art.113 bis - Collaborazioni Coordinate e Continuative
2. Art.140 - Controversie collettive
3. Art.119 - Trattenute sindacali
4. Art.41 - Aspettativa
5. Stagionalità
6. Art.62- Periodo di prova
7. Art.75 - Rinvio alla legge
8. Art.113 - Contrattazione aziendale
9. Art. 109 bis – Diritto alla disconnessione
10. Art.69 - Infortunio
11. Art.4 – Sicurezza sul luogo di lavoro, tutela della salute, formazione e rispetto dell’ambiente
12. Art.141 - Classificazione del personale
13. Art.128 - Organismo Paritetico Nazionale O.P.NA.S.P.
14. Art.120 - Ente Bilaterale autonomo del settore privato E.BI.A.S.P.
15. Art.49 - Provvedimenti disciplinari e procedure di licenziamento
... e altri 160 titoli
Trovati 175 titoli unici:

1. Art.113 bis - Collaborazioni Coordinate e Continuative
2. Art.140 - Controversie collettive
3. Art.119 - Trattenute si

In [4]:
# Query di test specifiche per il CCNL
test_queries = [
    "Cosa dice l'articolo 23 del CCNL?",  # Test per articolo specifico
    "Quali sono le norme su ferie e permessi?",  # Test per contenuto correlato
    "Come funziona l'orario di lavoro?",  # Test per argomento generale
    "Cosa dice il CCNL sui contratti a tempo determinato?",  # Test per tipo contratto
    "Retribuzione e inquadramento professionale",  # Test per argomento economico
    "Diritti e doveri dei lavoratori",  # Test per diritti
    "Formazione e aggiornamento professionale",  # Test per formazione
    "Query generica senza riferimenti specifici"  # Test controllo negativo
]

print("Query di test per CCNL:")
for i, query in enumerate(test_queries, 1):
    print(f"{i}. {query}")

print(f"\nTotale titoli disponibili per matching: {len(sample_titles)}")
if len(sample_titles) > 0:
    print(f"Primo titolo: {sample_titles[0]}")
    print(f"Ultimo titolo: {sample_titles[-1]}")

Query di test per CCNL:
1. Cosa dice l'articolo 23 del CCNL?
2. Quali sono le norme su ferie e permessi?
3. Come funziona l'orario di lavoro?
4. Cosa dice il CCNL sui contratti a tempo determinato?
5. Retribuzione e inquadramento professionale
6. Diritti e doveri dei lavoratori
7. Formazione e aggiornamento professionale
8. Query generica senza riferimenti specifici

Totale titoli disponibili per matching: 175
Primo titolo: Art.113 bis - Collaborazioni Coordinate e Continuative
Ultimo titolo: Art.74 - Finanziamento della formazione dell'apprendistato


In [5]:
# Test della funzione get_titles_from_query
print("=== RISULTATI DEI TEST ===")
print()

for i, query in enumerate(test_queries, 1):
    print(f"Test {i}: '{query}'")
    print("-" * 50)
    
    result = title_extractor.get_titles_from_query(query, sample_titles)
    
    print(f"Risultato: {result}")
    print(f"Titoli rilevanti: {result.get('relevant_titles', [])}")
    print(f"Ha match: {result.get('has_title_match', False)}")
    print(f"Confidenza: {result.get('confidence', 'N/A')}")
    print()
    print("=" * 60)
    print()

=== RISULTATI DEI TEST ===

Test 1: 'Cosa dice l'articolo 23 del CCNL?'
--------------------------------------------------
Risultato: {'relevant_titles': ['Art.23 - Conservazione del posto di lavoro'], 'has_title_match': True, 'confidence': 'high'}
Titoli rilevanti: ['Art.23 - Conservazione del posto di lavoro']
Ha match: True
Confidenza: high


Test 2: 'Quali sono le norme su ferie e permessi?'
--------------------------------------------------
Risultato: {'relevant_titles': ['Art.23 - Conservazione del posto di lavoro'], 'has_title_match': True, 'confidence': 'high'}
Titoli rilevanti: ['Art.23 - Conservazione del posto di lavoro']
Ha match: True
Confidenza: high


Test 2: 'Quali sono le norme su ferie e permessi?'
--------------------------------------------------
Risultato: {'relevant_titles': ['Art.19 - Ferie', 'Art.15 - Permessi retribuiti'], 'has_title_match': True, 'confidence': 'high'}
Titoli rilevanti: ['Art.19 - Ferie', 'Art.15 - Permessi retribuiti']
Ha match: True
Confidenz

In [ ]:
# Test con una query personalizzata
custom_query = input("Inserisci una query personalizzata (o premi Enter per saltare): ")

if custom_query.strip():
    print(f"\nTest query personalizzata: '{custom_query}'")
    print("-" * 50)
    
    result = title_extractor.get_titles_from_query(custom_query, sample_titles)
    
    print(f"Risultato: {result}")
    print(f"Titoli rilevanti: {result.get('relevant_titles', [])}")
    print(f"Ha match: {result.get('has_title_match', False)}")
    print(f"Confidenza: {result.get('confidence', 'N/A')}")
else:
    print("Nessuna query personalizzata inserita.")